# Genetic Algorithm Assignment
30% of the overall grade for this module

Marks indciated in sections below are based on percentage of marks allocated for this module

In this assignment you must choose a problem, and attempt to use the Genetic Alogrithm that we developed in class to solve this problem.





## The Problem         **(~30%)**

*   Description of the problem






Sudoku: A puzzle in which missing numbers are needed to be filled into a 9x9 grid of squares which are subdivided into 3x3 boxes so that every row, every column, and every box contains the numbers 1 through 9. There are different sizes of puzzle the most commom being the 9x9 grid but the size of the grid can vary depending on the difficulty of the puzzle. For this assignment we are only only focusing on 9x9 puzzles although this could be adapted for bigger/smaller puzzles easily enough

An example of a starting point for a puzzle might look like: 


<!-- TODO: -->

With the solution then becoming: 

<!-- ╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 8 │ 5 │ 1 ║ 3 │ 9 │ 2 ║ 4 │ 6 │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │ 2 │ 3 ║ 6 │ 1 │ 4 ║ 5 │ 8 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 4 │ 6 │ 9 ║ 5 │ 7 │ 8 ║ 1 │ 2 │ 3 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 9 │ 8 │ 6 ║ 1 │ 4 │ 7 ║ 3 │ 5 │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │ 7 │ 5 ║ 2 │ 6 │ 9 ║ 9 │ 1 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 4 │ 2 ║ 8 │ 5 │ 3 ║ 7 │ 9 │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 2 │ 3 │ 4 ║ 9 │ 8 │ 1 ║ 6 │ 7 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 1 │ 7 ║ 4 │ 2 │ 5 ║ 9 │ 3 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │ 9 │ 8 ║ 7 │ 3 │ 6 ║ 2 │ 4 │ 1 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝ -->




*   Discussion of the suitablity of Genetic Algorithms



#### Why is a genetic algorihm suitable?

By definition Genetic Algorithms are a search based-optimization technique frequently used to find optimal or near optimal solution to difficult problems https://www.tutorialspoint.com/genetic_algorithms/genetic_algorithms_introduction.htm

In the case of a sudoku puzzle, the algorithm can take in a starting point of a puzzle (Example defined above) and generate a population of potential solutions to the puzzle. These potential solutions can then be tested by the cost function to measure how close we are to the correct solution. Then using crossover and mutation functions we can generate new boards using the infomation gathered from the previous generations and discard boards that are not as good as the ones in the population. The overall goal of this is to eventually find boards that are deemed as the correct solution or as close as we can get given the starting point.

The cost function:

- Have a counter for invalidations
- Increase by one for each missing value (0).
- Increase by one for each repeated value in rows
- Increase by one for each repeated value in columns
- Increate by one for each repeated value in a 3x3 box


*   Complexity of the problem  (Overall marks allocated based on ..)

# The problem and the cost function   **(~20%)**

In [1445]:
#  insert code in ths cell
import numpy as np
import random
import copy
# Create the problem function to create a sudoku board

class sudoku_board:
    def __init__(self, board):
        self.initial_problem = copy.deepcopy(board)
        self.board = board
        self.rows = self.get_rows(board)
        self.columns = self.get_columns(board)
        self.boxes = self.get_boxes(board)
        self.cost = self.cost_function()

    def print_board(self, board):
        print('Current Board: ')
        for row in board:
            print(row)

    def update_board(self, board):
        self.board = board
        self.rows = self.get_rows(self.board)
        self.columns = self.get_columns(self.board)
        self.boxes = self.get_boxes(self.board)
        self.cost = self.cost_function()

    def get_rows(self, board):
        rows = []
        for row in board:
            rows.append(row)
        return rows

    def get_columns(self, board):
        columns = []
        for coulmns in range(9):
            columns.append([board[row][coulmns] for row in range(9)])
        return columns

    def get_boxes(self, board):
        boxes = []
        # range(start, stop, step) using syntax of range to get 0, 3rd, 6th as it should generate 
        # quick test of the range method
        for row in range(0, 9, 3):
            for column in range(0, 9, 3):
                # Using a list with a nested for loop to get all coordinates from the board
                # Ex. The first box generated is from 
                # 0,0 - 0,1, 0,2 - 1,0 - 1,1 - 1,2 - 2,0, 2,1
                # https://builtin.com/data-science/nested-list-comprehension-python
                # ? Checking Coordinates to see if they match up with what we expect
                # Row positions?
                # print([row+i for i in range(3)])
                # column positions
                # print([column+j for j in range(3)])

                box = [board[row+i][column+j] for i in range(3) for j in range(3)]
                boxes.append(box)
        return boxes
    
    def check_valid_row(self, row):
        if (len(row) == len(set(row))):
            return True
        else: 
            return False
    
    
    def cost_function(self):
        # Count Missing Values across the rows, columns, boxes and return a count of all invalidations across the board
        invalid_counts = 0
        for row in range(9):
            for index in range(9):
                if (self.rows[row][index] == 0):
                    invalid_counts += 1

        # Count repeats in rows
        for row in range(9):
            # https://stackoverflow.com/questions/49973739/python-how-to-remove-zeroes-from-a-list-in-python
            rows_with_empty_values_removed = list(filter(lambda num: num != 0, self.rows[row]))
            # Use a set to compare unique element list length to the currently list length with 0's removed 
            if (len(rows_with_empty_values_removed) != len(set(rows_with_empty_values_removed))):
                invalid_counts += 1

        # Count repeats in Columns
        for index in range(9):
            # https://stackoverflow.com/questions/49973739/python-how-to-remove-zeroes-from-a-list-in-python
            columns_with_empty_values_removed = list(filter(lambda num: num != 0, self.columns[index]))
            # Use a set to compare unique element list length to the currently list length with 0's removed
            if (len(columns_with_empty_values_removed) != len(set(columns_with_empty_values_removed))):
                invalid_counts += 1

        # Count repeats in grid boxes
        for index in range(9):
            # https://stackoverflow.com/questions/49973739/python-how-to-remove-zeroes-from-a-list-in-python
            boxes_with_empty_values_removed = list(filter(lambda num: num != 0, self.boxes[index]))
            # Use a set to compare unique element list length to the currently list length with 0's removed
            if (len(boxes_with_empty_values_removed) != len(set(boxes_with_empty_values_removed))):
                invalid_counts += 1

        return invalid_counts
        


    

In [1446]:
# Example Puzzles - 0 indicates

board =  [
[8, 5, 0, 0, 0, 2, 4, 0, 0],
[7, 2, 0, 0, 0, 0, 0, 0, 9],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 2],
[3, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 8, 0, 0, 7, 0],
[0, 1, 7, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 3, 6, 0, 4, 0]]

board2 =  [
[5, 0, 7, 2, 0, 0, 0, 9, 0],
[0, 0, 6, 0, 3, 0, 7, 0, 1],
[4, 0, 0, 0, 0, 0, 0, 6, 0],
[1, 0, 0, 4, 9, 0, 0, 0, 7],
[0, 0, 0, 5, 0, 8, 0, 0, 0],
[8, 0, 0, 0, 2, 7, 0, 0, 5],
[0, 7, 0, 0, 0, 0, 0, 0, 9],
[2, 0, 9, 0, 8, 0, 6, 0, 0],
[0, 4, 0, 0, 0, 9, 3, 0, 8]]

board3 =  [
[6, 0, 0, 0, 0, 9, 0, 0, 8],
[0, 8, 9, 5, 0, 0, 0, 1, 6],
[5, 0, 0, 0, 6, 0, 3, 0, 9],
[8, 3, 1, 0, 0, 0, 7, 0, 5],
[0, 2, 0, 0, 0, 0, 0, 6, 0],
[9, 0, 7, 0, 0, 0, 8, 4, 2],
[2, 0, 6, 0, 1, 0, 0, 0, 8],
[3, 7, 0, 0, 0, 6, 9, 2, 0],
[1, 0, 0, 3, 0, 0, 0, 0, 7]]

board4 =  [
[5, 0, 7, 2, 0, 0, 0, 9, 0],
[0, 0, 6, 0, 3, 0, 7, 0, 1],
[4, 0, 0, 0, 0, 0, 0, 6, 0],
[1, 0, 0, 4, 9, 0, 0, 0, 7],
[0, 0, 0, 5, 0, 8, 0, 0, 0],
[8, 0, 0, 0, 2, 7, 0, 0, 5],
[0, 7, 0, 0, 0, 0, 0, 0, 9],
[2, 0, 9, 0, 8, 0, 6, 0, 0],
[0, 4, 0, 0, 0, 9, 3, 0, 8]]

sample_solved_to_test_cost =  [
[4, 3, 5, 2, 6, 9, 7, 8, 1],
[6, 8, 2, 5, 7, 1, 4, 9, 3],
[1, 9, 7, 8, 3, 4, 5, 6, 2],
[8, 2, 6, 1, 9, 5, 3, 4, 7],
[3, 7, 4, 6, 8, 2, 9, 1, 5],
[9, 5, 1, 7, 4, 3, 6, 2, 8],
[5, 1, 9, 3, 2, 6, 8, 7, 4],
[2, 4, 8, 9, 5, 7, 1, 3, 6],
[7, 6, 3, 4, 1, 8, 2, 5, 9]]

s1 = sudoku_board(board)
# s1.print_board(board)

s1.cost_function()


65

# The Individual **(~30%)**


*   Chromosone
*   Crossover
*   Mutation



## Discussion and justification on the approaches taken for the above

---

The chromosome will intake a preset board to try and solve

- The mutation will take in to account the preset starting point of the puzzle and store the indices from the rows provided by the sudoku board function in a variable called - positions_of_non_preset_values_on_board this will be a list of indices of the non preset values from the puzzle, It will then insert random values in to the child board and return that board (I would have preferred to do this with the 3x3 boxes but didn't have enough time to write a method to regenerate the board using the grids)
- The crossover function will then based on the rate of change set within the parameters class will create a new child based on taking 2 parents and selecting a single point cross over with each non preset value taking a 50/50 chance to inherit the parent a or b's value. I chose to do a single point cross over as it seemed a more tidy way to track the evolution of randomness (https://www.tutorialspoint.com/genetic_algorithms/genetic_algorithms_crossover.htm)
- Parent selection will be a random sample of the population with the lowest cost function
- Population will be culled after max number of generations is reached based on cost function
---



In [1447]:
class individual:
    chromosome = None
    cost_of_chromosome = 0
    positions_of_non_preset_values_on_board = []

    def __init__(self, board_to_solve=None):
        if (board_to_solve is not None):
            self.chromosome = sudoku_board(board_to_solve)
            self.cost_of_chromosome = self.chromosome.cost_function()

    def get_positions_of_non_preset_values_in_box(self, list):
        # https://stackoverflow.com/questions/5419204/index-of-duplicates-items-in-a-python-list
        return [i for i, x in enumerate(list) if x == 0]
    
    def crossover(self, parent2):
        # Create a child from the initial problem
        child = self.chromosome.initial_problem
        # 50/50 chance to choose from either parent
        if random.random() < 0.5:
            # Remember we only want to select from the non preset values here
            for index in range(len(self.positions_of_non_preset_values_on_board)):
                for indexj in self.positions_of_non_preset_values_on_board[index]:
                    child[index][indexj] = self.chromosome.board[index][indexj]
        else: 
           # Remember we only want to select from the non preset values here
            for index in range(len(self.positions_of_non_preset_values_on_board)):
                for indexj in self.positions_of_non_preset_values_on_board[index]:
                    child[index][indexj] = parent2.chromosome.board[index][indexj]

        return child
        
    
    def mutate_board(self):
        current_chromosome_rows = self.chromosome.get_rows(self.chromosome.initial_problem)

        if not any(self.positions_of_non_preset_values_on_board):
                print('here')
                for index in range(len(current_chromosome_rows)):
                    # Only want to change non preset values from the puzzle
                    zero_positions_in_array = self.get_positions_of_non_preset_values_in_box(current_chromosome_rows[index])
                    self.positions_of_non_preset_values_on_board.append(zero_positions_in_array)

        if (self.chromosome.cost == 0):
            # Board is solved no need to return
            return self.chromosome
        else:
            # https://stackoverflow.com/questions/1593564/python-how-to-check-if-a-nested-list-is-essentially-empty
            
            # Randomise the non preset values on the board
            for index in range(len(self.positions_of_non_preset_values_on_board)):
                
                while (self.chromosome.check_valid_row(current_chromosome_rows[index]) == False):
                    for indexj in self.positions_of_non_preset_values_on_board[index]:
                        current_chromosome_rows[index][indexj] = random.randint(1, 9)                   
                    
            self.chromosome.update_board(current_chromosome_rows)
            return self.chromosome



In [1448]:
best_solution = individual(board_to_solve=board2)
best_solution.mutate_board()

best_solution.chromosome.board

here


[[5, 4, 7, 2, 8, 3, 1, 9, 6],
 [9, 5, 6, 2, 3, 4, 7, 8, 1],
 [4, 3, 2, 9, 8, 1, 5, 6, 7],
 [1, 5, 8, 4, 9, 3, 2, 6, 7],
 [1, 2, 7, 5, 6, 8, 3, 4, 9],
 [8, 9, 4, 3, 2, 7, 1, 6, 5],
 [5, 7, 3, 2, 8, 6, 1, 4, 9],
 [2, 3, 9, 4, 8, 5, 6, 1, 7],
 [6, 4, 2, 1, 7, 9, 3, 5, 8]]

In [1449]:

def choose_two_indices_from(max_number):
  ind1 = np.random.randint(max_number)
  ind2 = np.random.randint(max_number)
  if ind1 == ind2:
    return choose_two_indices_from(max_number)

  return ind1,ind2


## Running the algorithm  **(~10%)**

*   Parameter choices
*   Modifications (if any) to run_genetic
*   Rationale for the above



---

(Double click then ...)  Type Here...

---

There are only a few parameters needed in this case
- Number in population to preset a number of individuals to create
- Maximum number of children to generate
- parental selection rate to be able to adjust whether we take more from parent 1 or parent 2
- there is no real mutation rate as we are altering empty values with random values from 1-9  I suppose we could set one if the board were randmly generated with full values to determine how much to change

In [1450]:
#  Parameter class here
# Code for individual class in here 
class parameters:
    number_in_population = 100
    maximum_number_of_generations = 500
    parental_selection_rate = 0.5
    def __init__(self):
        self.number_in_population = 100
        self.maximum_number_of_generations = 500
        self.parental_selection_rate = 0.5
        self.child_rate = 0.6

In [1456]:
# Run Genetic method here

def run_genetic_algo(problem, parameters=parameters()):
    # read in problem
    cost = sudoku_board(problem).cost_function()

    population = []
    best_solution = individual(problem)
    best_solution.chromosome.cost = np.Infinity
    # Generate initial population, Params not working?????
    for i in range(500):
        new_individual = copy.deepcopy(individual(problem))
        population.append(new_individual)
        if (new_individual.cost_of_chromosome < best_solution.cost_of_chromosome):
            best_solution = copy.deepcopy(new_individual)

    for interation in range(100):
        children = []
    
        # Not sure why but cannot call from params so hard coding here
        while len(children) < 100 * 0.6:
            parent1_index, parent2_index = choose_two_indices_from(500)
            parent1 = population[parent1_index].mutate_board()
            parent2 = population[parent2_index].mutate_board()
            tempNewP1Individual = individual(parent1.board)
            tempNewP2Individual = individual(parent2.board)

            child1 = tempNewP1Individual.crossover(tempNewP2Individual)
            child2 = tempNewP2Individual.crossover(tempNewP1Individual)

            children.append(individual(child1))
            children.append(individual(child2))

    # add to population
    population += children
    
    # cull population
    population = sorted(population, key=lambda x: x.cost_of_chromosome)

    # cull population
    population = population[:500]

    if population[0].cost_of_chromosome < best_solution.cost_of_chromosome:
      best_solution = copy.deepcopy(population[0])

    # if (population[0].cost_of_chromosome != 0):
    #     run_genetic_algo(population[0].chromosome.board)

    print(best_solution.chromosome.cost, best_solution.chromosome.print_board(best_solution.chromosome.board))

In [1457]:
#  Running of the algorithm with outputs here
params = parameters()

run_genetic_algo(board2)

Current Board: 
[5, 6, 7, 2, 4, 8, 3, 9, 1]
[8, 2, 6, 9, 3, 5, 7, 4, 1]
[4, 3, 7, 5, 9, 8, 2, 6, 1]
[1, 3, 8, 4, 9, 6, 5, 2, 7]
[2, 7, 6, 5, 1, 8, 4, 3, 9]
[8, 4, 9, 3, 2, 7, 6, 1, 5]
[4, 7, 2, 6, 8, 1, 5, 3, 9]
[2, 1, 9, 7, 8, 3, 6, 5, 4]
[1, 4, 6, 5, 7, 9, 3, 2, 8]
17 None


In [1453]:
#  If changes to params or reruns of iterations dont overwrite, create more cells and copy code down to show evolution of final solution

## Results and conclusions    **(~10%)**

---

(Double click then ...)  Type Here...

---
I think if I had managed to do this via the 3x3 boxes to further increase the chances I could have got the cost much much lower, 
